### Cerință laborator 3 - Rețele Bayesiene

Un sistem de clasificare a e-mail-urilor folosește o rețea bayesiană pentru a evalua probabilitatea ca un e-mail să fie spam (S) în funcție de anumite caracteristici observate. Rețeaua include următoarele variabile:
##### S: E-mail-ul poate fi spam (S= 1) sau non-spam (S = 0).
##### O: E-mail-ul poate conține cuvântul "ofertă" (0 = 1) sau nu (0 = 0).
##### L: E-mail-ul poate conține link-uri (L= 1) sau nu (L= 0).
##### M: E-mail-ul poate avea o lungime mare (M = 1) sau nu (M = 0).
#### STRUCTURA REȚELEI BAYESIENE
Spam (S) influențează probabilitatea ca e-mail-ul să conțină cuvântul "ofertă" (O) și să conțină link-uri (L). Lungimea e-mail-ului (M) este influențată atât de faptul că este spam (S) cât și de prezența link-urilor (L).

### Probabilități
$$
\begin{aligned}
P(S = 1) &= 0.4, \\
P(S = 0) &= 0.6
\end{aligned}
$$
### Tabele de probabilități condiționate:
$$
\begin{aligned}
P(O = 1 \mid S = 0) &= 0.1, \\
P(O = 1 \mid S = 1) &= 0.7
\end{aligned}
$$
$$
\begin{aligned}
P(L = 1 \mid S = 1) &= 0.8, \\
P(L = 1 \mid S = 0) &= 0.3
\end{aligned}
$$
$$
\begin{aligned}
P(M = 1 \mid S = 1, L = 1) &= 0.9, \\
P(M = 1 \mid S = 0, L = 1) &= 0.6
\end{aligned}
$$
$$
\begin{aligned}
P(M = 1 \mid S = 1, L = 0) &= 0.5, \\
P(M = 1 \mid S = 0, L = 0) &= 0.2
\end{aligned}
$$

In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.independencies import Independencies
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = BayesianNetwork([('S', 'O'), ('S', 'L'), ('S', 'M'), ('L', 'M')])
cpd_S = TabularCPD(variable='S', variable_card=2, values=[[0.6], [0.4]])
cpd_O = TabularCPD(variable='O', variable_card=2, 
                   values=[[0.9, 0.3],  # P(O=0|S=0), P(O=0|S=1)
                           [0.1, 0.7]], # P(O=1|S=0), P(O=1|S=1)
                   evidence=['S'], evidence_card=[2])

cpd_L = TabularCPD(variable='L', variable_card=2, 
                   values=[[0.7, 0.2],  # P(L=0|S=0), P(L=0|S=1)
                           [0.3, 0.8]], # P(L=1|S=0), P(L=1|S=1)
                   evidence=['S'], evidence_card=[2])

cpd_M = TabularCPD(variable='M', variable_card=2,
                   values=[[0.8, 0.5, 0.4, 0.1],  # P(M=0|S,L)
                           [0.2, 0.5, 0.6, 0.9]], # P(M=1|S,L)
                   evidence=['S', 'L'], evidence_card=[2, 2])

model.add_cpds(cpd_S, cpd_O, cpd_L, cpd_M)

#assert model.check_model()
independencies = model.get_independencies()

print("Independențele din rețea:")
print(independencies)


D:\FII\PMP\Lab03\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Independențele din rețea:
(O ⟂ L, M | S)
(O ⟂ M | L, S)
(O ⟂ L | S, M)
(L ⟂ O | S)
(L ⟂ O | S, M)
(M ⟂ O | S)
(M ⟂ O | L, S)


Determinați cum clasifică rețeaua bayesiană e-mail-urile în funcție de atributele O, L și M. 

In [3]:

P_S = {1: 0.4, 0: 0.6}  # P(S=1) = 0.4, P(S=0) = 0.6
P_O_cond_S = {1: 0.7, 0: 0.1}  # P(O=1 | S=1) = 0.7, P(O=1 | S=0) = 0.1
P_L_cond_S = {1: 0.8, 0: 0.3}  # P(L=1 | S=1) = 0.8, P(L=1 | S=0) = 0.3
P_M_cond_SL = {(1, 1): 0.9, (0, 1): 0.6, (1, 0): 0.5, (0, 0): 0.2}  # P(M=1 | S, L)

def P_O_L_M_cond_S(S, O, L, M):
    # P(O, L, M | S)
    P_O = P_O_cond_S[S] if O == 1 else 1 - P_O_cond_S[S]
    P_L = P_L_cond_S[S] if L == 1 else 1 - P_L_cond_S[S]
    P_M = P_M_cond_SL[(S, L)] if M == 1 else 1 - P_M_cond_SL[(S, L)]
    
    return P_O * P_L * P_M

def P_O_L_M(O, L, M):
    P_O_L_M_S1 = P_O_L_M_cond_S(1, O, L, M) * P_S[1]
    P_O_L_M_S0 = P_O_L_M_cond_S(0, O, L, M) * P_S[0]
    return P_O_L_M_S1 + P_O_L_M_S0

def P_S_cond_O_L_M(S, O, L, M):
    P_O_L_M_S = P_O_L_M_cond_S(S, O, L, M) * P_S[S]
    P_O_L_M_total = P_O_L_M(O, L, M)
    return P_O_L_M_S / P_O_L_M_total


def classify_email(O, L, M):
    P_S1_cond_O_L_M = P_S_cond_O_L_M(1, O, L, M)
    P_S0_cond_O_L_M = P_S_cond_O_L_M(0, O, L, M)
    
    if P_S1_cond_O_L_M > P_S0_cond_O_L_M:
        return "Spam", P_S1_cond_O_L_M
    else:
        return "Non-Spam", P_S0_cond_O_L_M

for O in range(0,2):
    for L in range(0,2):
        for M in range(0,2):
            classification, probability = classify_email(O, L, M)
            print(f'[Pentru O={O},L={L} si M={M}] '
                  f'E-mail-ul este <{classification}> cu probabilitatea {probability:.4f}')


[Pentru O=0,L=0 si M=0] E-mail-ul este <Non-Spam> cu probabilitatea 0.9618
[Pentru O=0,L=0 si M=1] E-mail-ul este <Non-Spam> cu probabilitatea 0.8630
[Pentru O=0,L=1 si M=0] E-mail-ul este <Non-Spam> cu probabilitatea 0.8710
[Pentru O=0,L=1 si M=1] E-mail-ul este <Non-Spam> cu probabilitatea 0.5294
[Pentru O=1,L=0 si M=0] E-mail-ul este <Non-Spam> cu probabilitatea 0.5455
[Pentru O=1,L=0 si M=1] E-mail-ul este <Spam> cu probabilitatea 0.7692
[Pentru O=1,L=1 si M=0] E-mail-ul este <Spam> cu probabilitatea 0.7568
[Pentru O=1,L=1 si M=1] E-mail-ul este <Spam> cu probabilitatea 0.9492
